In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#df = pd.read_csv("localtime.csv", index_col=[0])
df = pd.read_csv("DF.csv", )

df = df.drop(['date', 'Connection','Accept-Encoding','CountryName'], 1)

#df = df.drop(['Accept','date', 'Connection','Accept-Encoding', 'Accept-Language','CountryName','City'], 1)
#df

In [2]:
df['path']=df['path'].str[:1]
#df = df.groupby(["IP","cookie","path"]).size().to_frame()


In [16]:
df2 = df.groupby(["IP","cookie","User-Agent","CountryCode","path","Accept-Language"]).size().reset_index(name="count")
df2 = df2
df2.head()

,IP,cookie,User-Agent,CountryCode,path,Accept-Language,count
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,4
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,lu,2
2,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,2
3,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,lu,2
4,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,lu,1


In [17]:
df_p = df2[df2['path']=="p"].rename(columns={"count": "p"}).drop(["path"],1)
df_k = df2[df2['path']=="k"].rename(columns={"count": "k"}).drop(["path"],1)

In [19]:
df_m = df_p.merge(df_k, on=['cookie','IP', 'User-Agent', 'CountryCode','Accept-Language'], how='outer')
df_m.head()

,IP,cookie,User-Agent,CountryCode,Accept-Language,p,k
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,NaN
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,2.0
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,1.0,2.0
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,5.0
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,2.0


In [20]:
def select_favorite(p,k):
    if pd.isna(p): 
        return "k"
    elif pd.isna(k):
        return "p"
    elif (p==k):
        return "eq"
    elif (p>k):
        return "p"
    else:
        return "k"
    

In [21]:
df_m['favorite'] = df_m.apply(lambda row: select_favorite(row['p'],row['k']), axis=1)
df_m.head()


,IP,cookie,User-Agent,CountryCode,Accept-Language,p,k,favorite
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,NaN,p
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,2.0,eq
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,1.0,2.0,k
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,2.0,5.0,k
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,4.0,2.0,p


In [22]:
df_m2 = df_m.drop(['IP', 'cookie','p', 'k'], 1)
df_m2


,User-Agent,CountryCode,Accept-Language,favorite
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p
1,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,eq
2,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,k
3,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,k
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,eq
6,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p
7,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p
8,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,lu,p


In [23]:
import pycountry_convert as pc

def country_to_cont(cc):
    cont_code = pc.country_alpha2_to_continent_code(cc)
    cont_name = pc.convert_continent_code_to_continent_name(cont_code)
    return cont_name





In [36]:
df_t = df_m2.loc[df_m2['CountryCode'].notna()].copy(deep=True)
df_t.head()
df_t['Continent'] = df_t.apply(lambda row: country_to_cont(row['CountryCode']), axis=1)
df_t = df_t.drop(['CountryCode'],1)
# df_t.to_csv("favorites.csv",index=False)
#df_t['localtime']=df_t['localtime'].str[:3]

In [35]:
df_t['IP']="*"
df_t['cookie']="*"
df_t['Accept-Language']="*"
df_t['city']="*"
df_t

,User-Agent,CountryCode,Accept-Language,favorite,IP,cookie,city
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*
1,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,eq,*,*,*
2,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,k,*,*,*
3,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,k,*,*,*
4,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*
5,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,eq,*,*,*
6,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*
7,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*
8,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*
9,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,*,p,*,*,*


In [12]:
# from pytz import country_timezones 
# df_c = df_t 
# df_c['datetime'] = df_c.apply(lambda x: country_timezones(x['CountryCode'])[0], axis=1)

In [45]:
df_t.groupby(['Continent','User-Agent','Accept-Language']).size().reset_index().sort_values(by=0)
#dups = df_t.pivot_table(index=['Continent','User-Agent','Accept-Language'], aggfunc='size')

,Continent,User-Agent,Accept-Language,0
35,Asia,Mozilla/5.0 (X11; Linux i686; rv:10.0) Gecko/2...,lt,120
20,Asia,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5...,z,142
156,South America,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,144
74,Europe,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,os,146
28,Asia,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:10...,jv,150
105,North America,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,157
21,Asia,Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-...,s,159
86,North America,Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AO...,ko,160
40,Asia,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,os,161
48,Europe,Mozilla/2.0 (compatible; MSIE 4.0; Windows 98),s,163


In [40]:
dups.sort_values(0).to_frame()

0
Continent     User-Agent                                         Accept-Language      
Asia          Mozilla/5.0 (X11; Linux i686; rv:10.0) Gecko/20... lt                120
              Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5;... z                 142
South America Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... s                 144
Europe        Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... os                146
Asia          Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:10.... jv                150
North America Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... s                 157
Asia          Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... s                 159
North America Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOL... ko                160
Asia          Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... os                161
Europe        Mozilla/2.0 (compatible; MSIE 4.0; Windows 98)     s                 163
North America Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... os                166
Asia          Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... v                 169
Europe        Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... v                 169
North America Mozilla/5.0 (iPhone; CPU iPhone OS 8_1 like Mac... ve                171
              Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit... mt                177
              Mozilla/5.0 (Android 4.4; Mobile; rv:41.0) Geck... ds                177
South America Mozilla/5.0 (Android 4.4; Mobile; rv:41.0) Geck... ds                178
Asia          Mozilla/5.0 (iPhone; CPU iPhone OS 8_1 like Mac... ve                178
South America Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; r... sl                183
Asia          Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; r... sl                192
Europe        Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... s                 196
Africa        Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US)... w                 197
Europe        Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit... v                 199
Africa        Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... h                 199
South America Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOL... ko                201
Europe        Mozilla/5.0 (Android; Mobile; rv:28.0) Gecko/28... io                201
North America Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... i                 203
              Mozilla/5.0 (Android; Mobile; rv:28.0) Gecko/28... io                214
              Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit... ig                218
Asia          Mozilla/5.0 (compatible; MSIE 10.0; Windows NT ... i                 221
...                                                                                ...
South America Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... mo                876
North America Mozilla/5.0 (SMART-TV; Linux; Tizen 2.3) AppleW... ky                878
Asia          MobileSafari/9537.53 CFNetwork/672.0.8 Darwin/1... sg                884
North America Mozilla/5.0 (Linux; U; Android 4.2; en-us; Nexu... v                 887
              Mozilla/5.0 (iPhone; CPU iPhone OS 6_0 like Mac... kv                892
Europe        Mozilla/5.0 (iPhone; U; CPU iPhone OS 5_1_1 lik... ss                892
South America Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... sg                909
Asia          Mozilla/5.0 (Linux; U; Android 4.2; en-us; Nexu... v                 939
North America Mozilla/5.0 (Android; Mobile; rv:28.0) Gecko/28... h                 945
              Mozilla/5.0 (Linux; U; Android-4.0.3; en-us; Ga... hi                946
              Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit... lu                971
Europe        Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-U... sg                972
North America Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US)... ff                981
Asia          Mozilla/5.0 (Windows; U; Windows NT 6.1; zh_CN)... sq               1

In [15]:
from pytz import country_timezones 
df_c = df_t 
df_c['datetime'] = df_c.apply(lambda x: x['datetime'].tz_convert(country_timezones(x['CountryCode'])[0]), axis=1)

KeyError: ('datetime', 'occurred at index 0')